In [1]:
import pandas as pd
import numpy as np
import ast
import json

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
dfproduction= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\CSVNUEVOS\Production.csv')

dfproduction.rename(columns={'id':'id_original'}, inplace= True)

dfproduction.head()

,id_original,production_companies,production_countries
0,862,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
1,8844,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
2,15602,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
3,31357,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
4,11862,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]"


In [4]:
dfproduction.shape

(44936, 3)

In [5]:
dfproduction.dropna(inplace=True)

In [6]:
dfproduction.shape

(44936, 3)

In [7]:
dfproduction['production_companies'] = dfproduction['production_companies'].apply(ast.literal_eval) 

In [8]:
df_explodedprod = dfproduction.explode('production_companies')

def extract_companies_info(x):
    if isinstance(x, dict):
        return pd.Series([x.get('name', ''),x.get('id', '')])
    else:
        return pd.Series(['', ''])

df_explodedprod[['name', 'id']] = df_explodedprod['production_companies'].apply(extract_companies_info) 

In [9]:
df_explodedprod.head()

,id_original,production_companies,production_countries,name,id
0,862,"{'name': 'Pixar Animation Studios', 'id': 3}","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",Pixar Animation Studios,3
1,8844,"{'name': 'TriStar Pictures', 'id': 559}","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",TriStar Pictures,559
1,8844,"{'name': 'Teitler Film', 'id': 2550}","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",Teitler Film,2550
1,8844,"{'name': 'Interscope Communications', 'id': 10201}","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",Interscope Communications,10201
2,15602,"{'name': 'Warner Bros.', 'id': 6194}","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",Warner Bros.,6194


In [10]:
columnas_eliminar= ['production_companies', 'production_countries', 'id']
df_explodedprod.drop(columns=columnas_eliminar, inplace=True)

In [11]:
df_explodedprod.shape

(81660, 2)

In [12]:
df_explodedprod.head()

,id_original,name
0,862,Pixar Animation Studios
1,8844,TriStar Pictures
1,8844,Teitler Film
1,8844,Interscope Communications
2,15602,Warner Bros.


In [13]:
df_explodedprod.to_csv('EstudiosIDpeli.csv', index=False)

In [14]:
df_explodedprod.shape

(81660, 2)

In [15]:
#df_explodedprod[['id_original','name']].groupby('name').value_counts()
## no le hago group by


In [ ]:
##teniendo en cuenta lo que me piden en la funcion de PRODUCTORAS, me traigo el data de movies para unirlo
## para obtener el revenue total

In [16]:
movies2= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\A-ETLconsignas\MoviesSMALL.csv')


In [18]:
movies3= movies2[['id','revenue']]
movies3.head()

,id,revenue
0,862,373554033.0
1,8844,262797249.0
2,15602,0.0
3,31357,81452156.0
4,11862,76578911.0


In [19]:
movies3.shape

(44936, 2)

In [20]:
productoras = pd.merge(df_explodedprod, movies3, left_on='id_original', right_on='id', how='left')

productoras.head()

,id_original,name,id,revenue
0,862,Pixar Animation Studios,862,373554033.0
1,8844,TriStar Pictures,8844,262797249.0
2,8844,Teitler Film,8844,262797249.0
3,8844,Interscope Communications,8844,262797249.0
4,15602,Warner Bros.,15602,0.0


In [23]:
##agrupo por nombre la cantidad de peliculas 
cantidad2= productoras[['id','name']].groupby('name').count().reset_index()
cantidad2.head()

,name,id
0,,11572
1,"""DIA"" Productions GmbH & Co. KG",1
2,# Andrea Sperling Productions,1
3,# Lexyn Productions,1
4,'A' Production Committee,1


In [25]:
##agrupo por nombre el revenue
ganancia2= productoras[['revenue','name']].groupby('name').sum().reset_index()
ganancia2.head()

,name,revenue
0,,1.757676e+09
1,"""DIA"" Productions GmbH & Co. KG",4.435093e+07
2,# Andrea Sperling Productions,0.000000e+00
3,# Lexyn Productions,0.000000e+00
4,'A' Production Committee,0.000000e+00


In [32]:
# uno ambos df
merged_ = pd.merge(cantidad2, ganancia2, on='name', how='left')
merged_.head(30)

,name,id,revenue
0,,11572,1.757676e+09
1,"""DIA"" Productions GmbH & Co. KG",1,4.435093e+07
2,# Andrea Sperling Productions,1,0.000000e+00
3,# Lexyn Productions,1,0.000000e+00
4,'A' Production Committee,1,0.000000e+00
5,(주)로드픽쳐스,1,0.000000e+00
6,(주)로제타시네마,1,0.000000e+00
7,(주)비에이엔터테인먼트,1,0.000000e+00
8,", Flaminia Produzioni Cinematografiche",1,0.000000e+00
9,", Planet Productions",1,0.000000e+00


In [30]:
##lo hago CSV
merged_.to_csv('ProductorasFuncion.csv', index=False)